# Package

In [1]:
import numpy as np
import pandas as pd

# Mesh

In [2]:
def mesh(h, lamb):
    k = lamb * h
    x = np.arange(-2, 5, h)
    x = np.append(x, np.array([5]))
    
    initial = initial_cond(x)
    
        
    t = np.arange(0, 1, k)
    t = np.append(t, np.array([1]))
    
    
    xx, tt = np.meshgrid(x, t)
    exact_mesh = xx - t[:, np.newaxis]
    
    u = np.zeros_like(xx)
    u[0, :] = initial
    return x, (u, exact_mesh)

In [3]:
def initial_cond(x):
    y = np.cos(np.pi * x) ** 2
    y[np.abs(x)>0.5] = 0
    return y

In [4]:
def exact_sol(mesh):
    y = np.cos(np.pi*mesh)**2
    for i in range(0, mesh.shape[0]):
        y[i,:][np.abs(mesh[i,:]) > 1/2] = 0
    return y


# Lax-Friedrichs

$$ \frac{v_{m}^{n+1} - \frac{1}{2}\left(v_{m+1}^{n}+v_{m-1}^{n}\right)}{k} + a\frac{v_{m+1}^{n} - v_{m-1}^{n}}{2h} $$
$$ v_{m}^{n+1} = \frac{1}{2}(1-\lambda)v_{m+1}^{n} + \frac{1}{2}(1+\lambda)v_{m-1}^{n}$$

In [5]:
def lax_fried(h, lamb=0.8):
    x, (u, exact) = mesh(h, lamb)
    exactsol = exact_sol(exact)
    for i in range(0,u.shape[0]-1):
        u[i, 0] = 0 ; u[i, -1] = 0
        u[i+1, 1:-1] = 1/2 * (1-lamb)*u[i, 2:] + (1/2)*(1+lamb)*u[i, 0:-2]
        
    error = (h * np.sum((u[-1,:]-exactsol[-1,:])**2))**0.5
    return error

# Lax-Wendroff
$$v_{m}^{n+1} = v_{m}^{n} - \frac{a}{2}\lambda (v_{m+1}^{n}-v_{m-1}^{n})+\frac{a^{2}}{2}\lambda^{2}(v_{m+1}-2v_{m}^{n}+v_{m-1}^{n})\\
a=1$$

In [6]:
def lax_wend(h, lamb=0.8):
    x, (u, exact) = mesh(h, lamb)
    exactsol = exact_sol(exact)
    for i in range(0,u.shape[0]-1):
        u[i, 0] = 0 ; u[i, -1] = 0
        u[i+1, 1:-1] = u[i, 1:-1] - 0.5*lamb*(u[i, 2:]-u[i, 0:-2])+0.5*(lamb**2)*(u[i,2:]-2*u[i,1:-1] + u[i, :-2])

    error = (h * np.sum((u[-1,:]-exactsol[-1,:])**2))**0.5
    return error

In [7]:
h = [0.1, 0.05, 0.025, 0.0125]
fried_error=[]
wend_error=[]
for i in h:
    fried_error.append(lax_fried(i))
    wend_error.append(lax_wend(i))

result={'h':h,'wendroff error':wend_error,'wendroff order':[2,2,2,2], 'friedrichs error':fried_error,'friedrichs order':[1,1,1,1]}

In [8]:
df1 = pd.DataFrame.from_dict(result)
df1.set_index("h", inplace=True)
df1.columns = [["Lax-Wendroff", "Lax-Wendroff", "Lax-Friedrichs","Lax-Friedrichs"], ['Error', 'Order', 'Error', 'Order']]
df1.head()

Lax-Wendroff       Lax-Friedrichs      
              Error Order          Error Order
h                                             
0.1000     0.058763     2       0.217857     1
0.0500     0.015824     2       0.114640     1
0.0250     0.004592     2       0.064999     1
0.0125     0.001343     2       0.035039     1